In [ ]:
import torchlensmaker as tlm
import torch
import torch.nn as nn


# how to get forward tree for sequential?
# make a SequentialKinematic and SequentialOptical? each forward is forward_kinematic and forward_optical
# add a top level Sequential to finish it off

# TransformTree is the tree of realized transforms in a sequential kinematic model
# It's a tree because sequential can contain other sequentials
# TransformTree : list[chain | TransformTree]

# KinematicSequencial
    # forward(self, chain) -> TransformTree
    
# OpticalSequencial
    # forward(self, TransformTree, Rays) -> Rays

# Sequential
    # forward(self, 


class SequentialKinematic(nn.Sequential, tlm.KinematicElement):
    def forward(self, chain) -> tuple[tlm.KinematicChain, tlm.KinematicChain]:
        joint = chain
        for mod in self:
            joint, chain = mod(chain)
        return joint, chain

"""
class SequentialOptical(nn.Sequential, tlm.OpticalElement):
    def forward(self, rays, tftree):

        # TransformTree[ko_indices[i]] is either:
        #  - the tf for optical element i
        #  - the TransformTree for sequential optical element i
        # in any case, we pass it to the optical element forward

        for mod in self:
            rays = mod(rays, tftree[ko_indices[i]])

        return rays
        """


def make_sequential(*elements):
    """
    Given a list of elements of any kind (kinematic, optical or sequential)
    make a sequential model with them in the order provided
    
    Returns a sequential model, i.e.:
        - a kinematic model
        - an optical model
        - list of indices indicating which kinematic element applies to each optical element
    """

    pass


class Sequential(nn.Module):
    def __init__(self, *elements):
        super().__init__()

        self.kinematic_model, self.optical_model, self.ko_indices = make_sequential(*elements)

    def forward(self, tf, rays):
        stuff = self.sequential_kinematic(data.transforms)
        out = self.sequential_optical(stuff)

        return out


    

model = SequentialKinematic(
    tlm.Gap(2),
    tlm.Gap(2),
    tlm.Rotate((15, 50)),
    tlm.Gap(2),
    tlm.Gap(1),
    #tlm.RefractiveSurface(tlm.Sphere(4.0, C=0.135327), material=tlm.NonDispersiveMaterial(1.517)),
)

tlm.show2d(model)

other design:


TODO
* Refactor optical surfaces into: AnchorSurface, CollisionSurface, Refraction/Reflection
* Isolate Artists into viewer.artists
* Move artist matching to elements artist() function
* Clean up render_sequence file and function
* Clean up principal plane code
* Implement sequential inversion: Sequential.reverse()

Elements have clean forward functions:
    kinematic: chain -> chain
    optical: (rays, tf) -> rays

ReflectiveSurface / RefractiveSurface are sequential element that combine:
AnchorSurface: chain -> (chain, chain)
CollisionSurface: (rays, tf) -> (t, normals, valid)
reflection / refraction
 
Inversion:
Given a sequential model, invert order and invert each element
Some elements like AbsolutePosition cannot be inverted


Previous design of fully splitting the chain is overly complex, especially for matching the optical elements to their tf. It's only useful compared to this simpler design if we ever want to invert the optical model while keeping the kinematic model the same. I don't see when I would need that, and if I do I can always to a custom non sequential model.


Idea: move rendering / artists (conversion to tlmviewer json format) to elements